In [2]:
import warnings
warnings.filterwarnings("ignore")
from experiment import Experiment, ExperimentData, ExperimentParams, ExperimentEvaluation
import pandas as pd
import numpy as np
import json
from libs.metrics import text_similarities, vector_distances
from libs.feature_extraction.vectorizers import vectorizers
import itertools

In [9]:
embedding_paths = json.load(open('embedding_paths.json', 'r'))
candidate_counts = [5]
similarity_list = list(text_similarities.keys())
distance_list = list(vector_distances.keys())

param_pairs = list(itertools.product(
    candidate_counts, similarity_list, distance_list))
params = []
for vectorizer_name, vectorizer in vectorizers.items():
    if vectorizer_name != 'bert':
        continue
    params.extend({'vectorizer': {"name": vectorizer_name, "args": {}},
                   'experiment_params': {
                        'candidate_count': param_pair[0],
                        'text_similarity': param_pair[1],
                        'vector_distance': param_pair[2],
                        'embeddings_path': embedding_paths[vectorizer_name],
                        'load_embeddings': True,
                    }}
                    for param_pair in param_pairs)


### Args from json

In [ ]:
# import json
# import glob

# E_experiment_params = glob.glob("./experiments/*E.json")

# params = []
# for param in E_experiment_params:
#     with open(param) as f:
#         params.append(json.load(f)) 

In [12]:
df = pd.read_csv("./data/comment_finder/all.csv")
for param in params:
    vectorizer = vectorizers[param["vectorizer"]["name"]](**param["vectorizer"]["args"])
    data = ExperimentData(df)
    exp_params = ExperimentParams(vectorizer, **param["experiment_params"])
    exp = Experiment(data, exp_params)
    res = exp.run()
    exp.save_results("./results")
    evaluate = ExperimentEvaluation(exp)
    evaluate.evaluate()
    evaluate.save_results("./results")     

Running experiment:  bert_cosine_bleu_C5


100%|██████████| 31/31 [03:31<00:00,  6.81s/it]


Running experiment:  bert_euclidean_bleu_C5


100%|██████████| 31/31 [04:22<00:00,  8.47s/it]


Running experiment:  bert_manhattan_bleu_C5


  3%|▎         | 1/31 [01:14<37:23, 74.78s/it]